In [12]:
# 6.2　グリッドサーチ２

In [13]:
from pathlib import Path
import shutil
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [14]:
#mnist-train.py

In [15]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# 再現性の確保のために乱数シードを固定（数値は何でもよい）
tf.set_random_seed(12345)

# 入力データ
# MNISTのワンホット表現での読み込み
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, validation_size=10000)

# 0 入力画像
x = tf.placeholder(tf.float32, name='x')

# 1 サイズ変更
x_1 = tf.reshape(x, [-1, 28, 28, 1])

# 2 畳み込み
# ランダムカーネル
k_0 = tf.Variable(tf.truncated_normal([4, 4, 1, 10], mean=0.0, stddev=0.1))
# 畳み込み
x_2 = tf.nn.conv2d(x_1, k_0, strides=[1, 3, 3, 1], padding='VALID')

# 3 活性化関数
x_3 = tf.nn.relu(x_2)

# 4 プーリング
x_4 = tf.nn.max_pool(x_3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

# 5 サイズ変更
x_5 = tf.reshape(x_4, [-1, 160])

#ドロップアウト付きの全結合
def matmul_plus_bias_with_dropout(x, w, b, p):
    return tf.matmul(tf.nn.dropout(x, keep_prob=p), w) + b

# 6 全結合
# 重みとバイアス
w_1 = tf.Variable(tf.zeros([160, 40]))
b_1 = tf.Variable([0.1] * 40)
#ドロップアウト率
p_1 = tf.placeholder(tf.float32, name='p_1')
# 全結合
x_6 = matmul_plus_bias_with_dropout(x_5, w_1, b_1, p_1)

# 7 活性化関数
x_7 = tf.nn.relu(x_6)

# 8 全結合
# 重みとバイアス
w_2 = tf.Variable(tf.zeros([40, 10]))
b_2 = tf.Variable([0.1] * 10)
#ドロップアウト率
p_2 = tf.placeholder(tf.float32, name='p_2')
# 全結合
x_8 = matmul_plus_bias_with_dropout(x_7, w_2, b_2, p_2)

# 9 確率化
y = tf.nn.softmax(x_8, name='y')

# 10 損失関数の最小化
# 正解ラベル
labels = tf.placeholder(tf.float32, name='labels')
# 損失関数（交差エントロピー）と最適化処理（Adam）
loss = -tf.reduce_sum(labels * tf.log(y)) / tf.cast(tf.shape(y)[0], tf.float32)
optimizer = tf.train.AdamOptimizer().minimize(loss)

# 11 精度検証 (Top-1 accuracy)
prediction_match = tf.equal(tf.argmax(y, axis=1), tf.argmax(labels, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction_match, tf.float32), name='accuracy')

# 損失関数と精度の出力をトレース対象とする
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)
summary = tf.summary.merge([loss_summary, accuracy_summary])

# パラメーター
# バッチサイズ
BATCH_SIZE = 32
# 学習回数
NUM_TRAIN = 10_000
# 学習中の出力頻度
OUTPUT_BY = 500
#ドロップアウト率
DROPOUT_PROB_1 = 0.8
DROPOUT_PROB_2 = 0.8

# 以前のログが残っていれば削除する（通常は残すようにしてください）
logdir = Path('logs/log3')
if logdir.exists():
    shutil.rmtree(str(logdir))

# 学習の実行
# TensorBoard上でグラフを重ねて描くために、2つのwriterを使用する
with tf.summary.FileWriter('logs/log3/train') as train_writer, \
    tf.summary.FileWriter('logs/log3/val') as val_writer:
    sess.run(tf.global_variables_initializer())
    dropout_prob = {p_1: DROPOUT_PROB_1, p_2: DROPOUT_PROB_2}
    saver = tf.train.Saver()
    for i in range(NUM_TRAIN):
        batch = mnist.train.next_batch(BATCH_SIZE)
        inout = {x: batch[0], labels: batch[1]}
        if i % OUTPUT_BY == 0:
            train_loss, train_accuracy, train_summary = \
                sess.run([loss, accuracy, summary], feed_dict={**inout, p_1: 1.0, p_2: 1.0})
            val_loss, val_accuracy, val_summary = \
                sess.run([loss, accuracy, summary], feed_dict={
                x: mnist.validation.images, labels: mnist.validation.labels, p_1: 1.0, p_2: 1.0})
            print('step {:d}, train_loss {:.2f}, val_loss {:.2f}, train_acc {:.2f}, val_acc {:.2f}'.format(
                i, train_loss, val_loss, train_accuracy, val_accuracy))
            # ログの出力
            train_writer.add_summary(train_summary, global_step=i)
            val_writer.add_summary(val_summary, global_step=i)
            # 過程の保存
            saver.save(sess, 'model3/my-model', global_step=i)
            # ファイルへの強制書き出し
            train_writer.flush()
            val_writer.flush()
        optimizer.run(feed_dict={**inout, **dropout_prob})

# 最終結果の保存
saver.save(sess, 'model3/my-model')

C:\Users\210450279\Anaconda3\envs\rebases\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, train_loss 2.30, val_loss 2.30, train_acc 0.12, val_acc 0.10
step 500, train_loss 1.01, val_loss 0.80, train_acc 0.72, val_acc 0.76
step 1000, train_loss 0.49, val_loss 0.51, train_acc 0.84, val_acc 0.86
step 1500, train_loss 0.36, val_loss 0.38, train_acc 0.88, val_acc 0.89
step 2000, train_loss 0.18, val_loss 0.32, train_acc 0.97, val_acc 0.91
step 2500, train_loss 0.30, val_loss 0.30, train_acc 0.88, val_acc 0.91
step 3000, train_loss 0.44, val_loss 0.26, train_acc 0.88, val_acc 0.92
step 3500, train_loss 0.18, val_loss 0.24, train_acc 0.97, val_acc 0.93
step 4000, train_loss 0.17, val_loss 0.23, train_acc 0.97, val_acc 0.94
step 4500, train_loss 0.11, val_loss 0.21, train_acc 0.97, val_acc 0.94
step 5000, train_loss 0.17, val_loss 0.20, train_acc 0.97, val_acc 0.94
step 5500, train_l

'model3/my-model'

In [16]:
#mnist-test.py

In [18]:
# モデルの読み込み
saver = tf.train.import_meta_graph('model3/my-model.meta')

# データフローグラフからプレースホルダーと精度測定処理ノードを取得
graph = tf.get_default_graph()
x = graph.get_tensor_by_name('x:0')
labels = graph.get_tensor_by_name('labels:0')
p_1 = graph.get_tensor_by_name('p_1:0')
p_2 = graph.get_tensor_by_name('p_2:0')
y = graph.get_tensor_by_name('y:0')
accuracy = graph.get_tensor_by_name('accuracy:0')

with tf.Session() as sess:
    # 変数初期化
    sess.run(tf.global_variables_initializer())

    # モデル読み込み
    saver.restore(sess, 'model3/my-model')

    # テストデータによる精度検証
    test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, labels: mnist.test.labels, p_1: 1.0, p_2: 1.0})
    print('test accuracy {:.2f}'.format(test_accuracy))


INFO:tensorflow:Restoring parameters from model3/my-model
test accuracy 0.96
